# Case 3 - Fluent.io
* Kyle Anderson

## Problem:
* Fluent has found that their language learning app is price inelastic, which means users are less sensitive to price changes. This means that Fluent has the ability to capture more revenue by increasing prices from their current Plus 6.99/month plan. However, there is no long going to be a singular price. The proposed packages are Basic (7.99/month), Pro (9.99/month) and Premium $16.99/month). Within these packages are varying features that will be more customized to users who want more or less features within the app. The goal of these three tiers are to retain their customer base but take advantage of their price insensitive users and capture more revenue or Average Revenue per User.

## Technical Goal:

* The technical goal of the case, is to run a A/B/C/D test to evaluate the impact of three new pricing tiers. We are using on Average Revenue Per User (ARPU) to measure the outcome because it captures both conversion rates and customer spending to view the impact on revenue.

## Load Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import scipy.stats as stats
from scipy.stats import norm
from math import sqrt
import warnings

## Load Data

In [2]:
fluent_hist = fr'https://raw.githubusercontent.com/jefftwebb/data/main/fluent_historical.csv'
fluent_df = pd.read_csv(fluent_hist, index_col=False)

fluent_dict = fr'https://raw.githubusercontent.com/jefftwebb/data/main/fluent_historical_dictionary.csv'
fluent_dict_df = pd.read_csv(fluent_dict, index_col=False)

In [3]:
### Inspect Dictionary for Context

In [4]:
fluent_dict_df.head(10)

,variable,type,description
0,customer_id,character,Unique identifier for each customer.
1,plan,character,Type of subscription plan (free or paid).
2,date,date,The transaction date. For free subscriptions ...
3,paid,integer,The amount spent on either the monthly subscri...
4,type,character,The type of trasnaction: subscription or purc...


### Inspect Fluent Historical Data

In [5]:
fluent_df.tail()
# We see customer_id, current plan, payments made and when, and the type of transaction

,customer_id,plan,date,paid,type
86223,EZ6513,paid,2023-12-25,6.99,subscription
86224,VL6409,paid,2023-12-28,6.99,subscription
86225,DL66338,paid,2023-12-30,6.99,subscription
86226,IK61234,paid,2023-12-30,6.99,subscription
86227,LD43645,paid,2023-12-31,6.99,subscription


In [6]:
# Show all types of transactions
print('Unique transaction event types')
print(fluent_df['type'].unique())

Unique transaction event types
['purchase' 'subscription']


In [7]:
# Show all types of plans
print('Unique plans')
print(fluent_df['plan'].unique())

Unique plans
['paid' 'free']


## Question 1:
#### To Measure Conversion
* All customers start on the free plan when they register for the app and have the option to upgrade to paid, which is known as a conversion. Both plans, free and paid, offer in-app purchases for additional services.

In [8]:
df_f = fluent_df.copy()

plan_counts = df_f['plan'].value_counts()
print('Unique plan counts')
print(plan_counts)
print()

total_plans = len(df_f['plan'])
print('Unique total plans')
print(total_plans)

paid_ratio = df_f[df_f['plan']=='paid'].shape[0] / total_plans
print('Paid ratio')
print(round(paid_ratio, 2))
free_ratio = df_f[df_f['plan']=='free'].shape[0] / total_plans
print('Free ratio')
print(round(free_ratio, 2))

Unique plan counts
plan
paid    55604
free    30624
Name: count, dtype: int64

Unique total plans
86228
Paid ratio
0.64
Free ratio
0.36


### To get true conversion, we need to find the users who went from free to paid

In [9]:
warnings.filterwarnings("ignore", category=DeprecationWarning)


def check_conversion(group):
    if 'free' in group['plan'].values and 'paid' in group['plan'].values:
        return 1
    return 0

df_f['converted'] = df_f.groupby('customer_id').apply(check_conversion).reset_index(drop=True)

total_free_users = df_f[df_f['plan'] == 'free']['customer_id'].nunique()
total_converted_users = df_f[df_f['converted'] == 1]['customer_id'].nunique() # Users converted from free to paid

conversion_rate = total_converted_users / total_free_users
conversion_rate = round(conversion_rate, 2)

print(f"Total Free Users")
print(total_free_users)
print(f"Total Converted Users")
print(total_converted_users)
print(f"Conversion Rate")
print(conversion_rate)

# The answer is that NONE of the people in the data converted from free to paid

Total Free Users
24111
Total Converted Users
0
Conversion Rate
0.0


In [10]:
np.random.seed(93)
# sample
n = 1000
# 20% conversion
conversion_prob_1 = 0.2
# 5% conversion for a more conservative approach
conversion_prob_2 = 0.05
sim_spend = 10
sim_std = 2 # low variability
# spending_mean = df_f['paid'].mean()
# print(spending_mean)
# spending_std = df_f['paid'].std()
# print(spending_std)


def simulate_arpu(conversion_prob, n, sim_spend, sim_std):
    conversions = np.random.binomial(1, conversion_prob, n)
    # random spending
    spending = np.random.normal(sim_spend, sim_std, n)
    # ARPU
    arpu = np.mean(conversions * spending)
    return arpu


arpu_1 = simulate_arpu(conversion_prob_1, n, sim_spend, sim_std)
arpu_2 = simulate_arpu(conversion_prob_2, n, sim_spend, sim_std)


print(fr"ARPU with {round(conversion_prob_1,2)*100}% conversion rate: ${arpu_1:.2f}")
print(fr"ARPU with {round(conversion_prob_2,2)*100}% conversion rate: ${arpu_2:.2f}")


ARPU with 20.0% conversion rate: $1.86
ARPU with 5.0% conversion rate: $0.55


## Answer 1
* As shown, more conversion means more ARPU as it is a linear scale. I chose to go more conservative on the conversion to show a worst case scenario. However, using the actual mean spend and standard deviation shows higher ARPU. The actual mean and standard deviation are 13 and 20. This standard deviation shows high variability and differences in spend in the individuals which further supports the case for tiered pricing.

## Question 2
* Get 2022 data, monthy stdv revenue per user (SDRPU)

#### Get 2022 data

In [11]:
warnings.filterwarnings("ignore")
df_f['year'] = pd.to_datetime(df_f['date'], errors='coerce')
df_f22 = df_f[df_f['year'].dt.year == 2022]
df_f22['year'] = 2022

#### Get monthy SDRPU

In [12]:
warnings.filterwarnings("ignore")

df_f22['date'] = pd.to_datetime(df_f22['date'], errors='coerce')
df_f22['month'] = df_f22['date'].dt.to_period('M')
# Get monthy revenue and signify customer as that will capture the mean and variability
month_rev = df_f22.groupby(['customer_id','month'])['paid'].sum().reset_index()
print("Monthly revenue per customer")
print(month_rev.head(2))
print()
monthly_stats = month_rev.groupby('month').agg(ARPU=('paid', 'mean'),SDRPU=('paid', 'std')).reset_index()
monthly_stats.head(13)

Monthly revenue per customer
  customer_id    month  paid
0     AA11573  2022-07  6.99
1     AA11573  2022-08  6.99



,month,ARPU,SDRPU
0,2022-01,19.701100,29.423394
1,2022-02,18.917938,27.852079
2,2022-03,18.045778,27.403972
3,2022-04,16.717551,26.085455
4,2022-05,16.789353,25.879768
5,2022-06,16.126559,25.133832
6,2022-07,15.790107,24.759608
7,2022-08,16.052698,25.189190
8,2022-09,15.334967,24.231893
9,2022-10,15.931247,24.901931


## Answer 2
* Perhaps this study was started because of the declining ARPU. The variability remains high across all months but is not consistent.

## Question 3
* Get sample size for ABCD test with MEI of 2 dollars and SDRPU of 24 dollars.

In [13]:
# Get historical ARPU
customer_revenue = df_f.groupby('customer_id')['paid'].sum().reset_index()
total_arpu = customer_revenue['paid'].mean()
print("Historical ARPU")
print("$", round(total_arpu, 2))

Historical ARPU
$ 38.8


In [14]:
# Get historical conversion rate, which shown above is 0 and 0%
free_users = set(df_f[df_f['plan'] == 'free']['customer_id'])
paid_users = set(df_f[df_f['plan'] == 'paid']['customer_id'])


converted_users = free_users.intersection(paid_users)
num_converted = len(converted_users)

num_free_users = len(free_users)

# conversion
conversion_rate = (num_converted / num_free_users) * 100 if num_free_users > 0 else 0
conversion_rate

0.0

In [17]:
sdrpu = 24
mei = 2
# p2 = sdrpu + 2 # MEI
alpha = 0.05 # set by assignment
power = 0.8 # set by assignment

# Z1alpha
z_alpha = norm.ppf(1 - alpha/2) # 2 tail
# Z1beta
z_beta = norm.ppf(power)

p_bar = (sdrpu + p2) / 2 # Pbar is the overall conversion rate

# n = (((z_alpha * sqrt(2 * p_bar * (1 - p_bar))) + sqrt(sdrpu * (1 - sdrpu) + p2 * (1 - p2)) * z_beta)/ (p2 - sdrpu)) ** 2
n = ((z_alpha + z_beta) ** 2 * sdrpu ** 2) / (mei ** 2)
n_per_group = np.ceil(n)
total_sample_size = int(n_per_group * 4)

print(f"Sample size required per group: {n_per_group}")
print(f"Total sample size required: {total_sample_size}")

# Numpy ceiling to get whole customer
# sample = np.ceil(n)
# print("Sample Size:")
# sample

Sample size required per group: 1131.0
Total sample size required: 4524
